## Operations

In [24]:
import numpy as np

In [1]:
class Operation():
    
    def __init__(self, input_nodes=[]):
        self.input_nodes = input_nodes
        self.output_nodes  = []
        
        for node in input_nodes:
            node.output_nodes.append(self)
            
        _default_graph.operations.append(self)
    
    
    def compute(self):
        pass

In [2]:
class add(Operation): # naming with small letter to mimic tensorflow
    
    def __init__(self, x, y):
        super().__init__([x,y])
        
    def compute(self, x_var, y_var):
        self.inputs = [x_var, y_var]
        return x_var + y_var

In [3]:
class multiple(Operation):
    
    def __init__(self, x, y):
        super().__init__([x, y])
        
    def compute(self, x, y):
        self.inputs = [x, y]
        return x * y

In [4]:
class matmul(Operation):
    
    def __init__(self, x, y):
        super().__init__([x, y])
        
    def compute(self, x, y):
        self.inputs = [x, y]
        return x.dot(y)

## Placeholders and Variables

Placeholder is an empty node that needs a value to be provided to compute output

Variable is already filled in with changeable parameters of the graph. Like the weights.

Graph is the global parameter that connects the variables and placeholders to the operations.

In [5]:
class Placeholder():
    def __init__(self):
        self.output_nodes = []
        # grab the global variables for the default graph
        _default_graph.placeholders.append(self)

In [6]:
class Variable():
    def __init__(self, initial_value=None):
        self.value = initial_value
        self.output_nodes = []
        _default_graph.variables.append(self)

In [7]:
# Graph connects placeholder and variables to the operations they belong to
class Graph():
    def __init__(self):
        self.operations = []
        self.placeholders = []
        self.variables = []
        
    def set_as_default(self):
        global _default_graph # this makes us be able to access it in Placeholder, Variable
                                # and Operations class
        _default_graph = self

Now we have operations, placeholders and variables and we have a graph that keeps track of them all

z = Ax + b

A = 10

b = 1

z = 10x + 1

In [8]:
g = Graph()
g.set_as_default()
A = Variable(10)
b = Variable(1)
x = Placeholder()

In [9]:
y = multiple(A, x)

In [10]:
y

In [11]:
z = add(y, b)

In [12]:
z

Actual muliplication and addition never happened

We need to add a traverse post order function to do that in the correct order

## Creating a Session

Make a post order traversal function for correct precedence

In [13]:
def traverse_postorder(operation):
    nodes_postorder = []
    def recurse(node):
        if isinstance(node, Operation):
            for input_node in node.input_nodes:
                recurse(input_node)
        nodes_postorder.append(node)
        
    recurse(operation)
    return nodes_postorder

In [14]:
class Session():
    # The opperation itself will be the operation to compute
    # The feed dictionary maps the placeholders to the value
    def run(self, operation, feed_dict={}):
        nodes_postorder = traverse_postorder(operation)
        for node in nodes_postorder:
            if type(node) == Placeholder:
                node.output = feed_dict[node]
            elif type(node) == Variable:
                node.output = node.value
            else: # Operation
                node.inputs = [input_node.output for input_node in node.input_nodes]
                node.output = node.compute(*node.inputs)
            if type(node.output) == list:
                node.output = np.array(node.output)
        return operation.output

Lets execute this

In [15]:
sess = Session()

In [16]:
result = sess.run(operation=z, feed_dict={x:10})

In [17]:
result

101

Trying matrix operations

In [20]:
g = Graph()
g.set_as_default()

A = Variable([[10, 20], [30, 40]])
b = Variable([1, 2])

x = Placeholder()
y = matmul(A, x)
z = add(y, b)

In [22]:
sess = Session()

In [25]:
sess.run(operation=z, feed_dict={x:10})

array([[101, 202],
       [301, 402]])